In [ ]:
import sys

eff_root = "/home/haridas/projects/opensource/automl/efficientdet/"
sys.path.insert(0, eff_root)

In [ ]:
import numpy as np
import pandas as pd
import os

from PIL import Image


# efficientdet related packages.
import tensorflow.compat.v1 as tf
import hparams_config
import dataloader
import det_model_fn

from keras import train, train_lib


os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
# print_tensors_in_checkpoint_file(file_name="/home/haridas/projects/opensource/automl/efficientdet/efficientdet-d1/", tensor_name="resample_p1", all_tensors=True)

## TFRecord analysis

In [ ]:
tfr_file = "/home/haridas/projects/opensource/automl/efficientdet/data/val/val-00000-of-00001.tfrecord"

In [ ]:
raw_dataset = tf.data.TFRecordDataset(tfr_file)

In [ ]:
raw_record = raw_dataset.take(1)

In [ ]:
example = tf.train.Example()

In [ ]:
# example.ParseFromString(raw_record.numpy())

## Model Inference Experiment

In [ ]:
min_score_threh = 0.35
max_boxes_to_draw = 100
line_thickness = 2

In [ ]:
img_path = "/home/haridas/projects/mystique/data/pic2card_dataset_01_apr_2021/test/2.png"

In [ ]:
img = Image.open(img_path)
img.size

In [ ]:
# !python /home/haridas/projects/opensource/automl/efficientdet/model_inspect.py --help

# tf.estimator.tpu.RunConfig(
#     model_dir="path",
#     evaluation_master="",
#     cluster=None
# )

## Model Inspect

In [ ]:
checkpoint = tf.train.get_checkpoint_state(os.path.join(eff_root, "out/efficientdet-d1-finetuned-2020-11-04-1604487101/"))

In [ ]:
checkpoint.model_checkpoint_path

In [ ]:
model_fn_instance = det_model_fn.get_model_fn("efficientdet-d1")

In [ ]:
config = hparams_config.get_detection_config("efficientdet-d1")

In [ ]:
params = dict(config.as_dict(),
              model_name="efficientdet-d1",
              iterations_per_loop=100,
              model_dir=checkpoint.mo0jd
             )

## Saved Model Inspection

In [ ]:
saved_model_path = "/home/haridas/projects/opensource/automl/efficientdet/out/efficientdet-d1-finetuned-2021-04-29-1619710132/saved_model"

In [ ]:
imported.signatures["serving_default"]

In [ ]:
# imported = tf.saved_model.load_v2(saved_model_path)

## Frozen Model Inference

The model is trained with custom configuration and augmentation, the changes are commited here: https://github.com/haridas/automl/tree/pic2card

Reference: https://github.com/google/automl/issues/933


Export the saved models from the training checkpoints using the following command,

```bash
$ python model_inspect.py --runmode=saved_model \
    --model_name=efficientdet-d1 \
    --ckpt_path out/efficientdet-d1-finetuned-2021-04-29-1619710132 \
    --saved_model_dir out/efficientdet-d1-finetuned-2021-04-29-1619710132/
```

Above command will save both `saved_model` and `frozen_graph` version of the trained checkpoint.
We are using frozen_graph for our deployments.

In [ ]:
frozen_model_path = "/home/haridas/projects/opensource/automl/efficientdet/out/efficientdet-d1-finetuned-2021-04-29-1619710132/saved_model/efficientdet-d1_frozen.pb"

In [ ]:
def get_image_array(im_path):
    im = Image.open(im_path).convert("RGB")
    im_arr = np.frombuffer(im.tobytes(), dtype=np.uint8)
    im_arr = im_arr.reshape((1, im.size[1], im.size[0], 3))jkk
    return im_arr

In [ ]:
with tf.io.gfile.GFile(frozen_model_path, 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    loaded = graph_def.ParseFromString(f.read())

In [ ]:
def wrap_frozen_graph(graph_def, inputs, outputs, print_graph=False):
    def _imports_graph_def():
        tf.compat.v1.import_graph_def(graph_def, name="")

    wrapped_import = tf.compat.v1.wrap_function(_imports_graph_def, [])
    import_graph = wrapped_import.graph
    
#     if print_graph == True:
#         print("-" * 50)
#         print("Frozen model layers: ")
#         layers = [op.name for op in import_graph.get_operations()]
#         for layer in layers:
#             print(layer)
#         print("-" * 50)

    return wrapped_import.prune(
        tf.nest.map_structure(import_graph.as_graph_element, inputs),
        tf.nest.map_structure(import_graph.as_graph_element, outputs))

In [ ]:
frozen_func = wrap_frozen_graph(graph_def=graph_def,
                                inputs=["image_arrays:0"],
                                outputs=["detections:0"]
                               )

In [ ]:
# get_image_array(img_path).shape

In [ ]:
detections = frozen_func(tf.constant(get_image_array(img_path)))[0].numpy().squeeze()

In [ ]:
detections[:, 5]

# Keras implementation of EfficientDet

In [ ]:
config_d1 = hparams_config.get_detection_config("efficientdet-d1")

In [ ]:
config_d1.model_dir

In [ ]:
train_lib.EfficientDetNetTrain(config=config)

In [ ]:
# with tf.Session(graph=tf.Graph()) as sess:
#     saver = tf.train.import_meta_graph(checkpoint.model_checkpoint_path + '.meta', clear_devices=True)

# References

1. [https://ml6.eu/a-practical-guide-to-fine-tuning-efficientdet-for-object-detection/](https://ml6.eu/a-practical-guide-to-fine-tuning-efficientdet-for-object-detection/)